##### Import packages and setup path

In [9]:
import sys
import os
import yfinance as yf
import pandas as pd
import numpy as np
from textblob import TextBlob


current_dir=os.getcwd()                     #current working directory

parent_dir=os.path.dirname(current_dir)     # Get the parent directory
sys.path.append(parent_dir)                    # append to the lookup directory list

from scripts.financial_analyzer import PlotGraph, CommonAnalysis, FinancialAnalyzer, SentimentAnalysis

##### Load dataset and create a Dataframe

In [10]:
# Load Historical csv data into a dataframe
df_news_report=CommonAnalysis.read_csv_file('../dataset/raw_analyst_ratings.csv')

df_AAPL_historical=CommonAnalysis.read_csv_file('../dataset/yfinance_data/AAPL_historical_data.csv')
df_AMZN_historical=CommonAnalysis.read_csv_file('../dataset/yfinance_data/AMZN_historical_data.csv')
df_GOOG_historical=CommonAnalysis.read_csv_file('../dataset/yfinance_data/GOOG_historical_data.csv')
df_META_historical=CommonAnalysis.read_csv_file('../dataset/yfinance_data/META_historical_data.csv')
df_MSFT_historical=CommonAnalysis.read_csv_file('../dataset/yfinance_data/MSFT_historical_data.csv')
df_NVDA_historical=CommonAnalysis.read_csv_file('../dataset/yfinance_data/NVDA_historical_data.csv')
df_TSLA_historical=CommonAnalysis.read_csv_file('../dataset/yfinance_data/TSLA_historical_data.csv')

# Convert the 'date' column to datetime
df_AAPL_historical['Date'] = pd.to_datetime(df_AAPL_historical['Date'])
df_AMZN_historical['Date'] = pd.to_datetime(df_AMZN_historical['Date'])
df_GOOG_historical['Date'] = pd.to_datetime(df_GOOG_historical['Date'])
df_META_historical['Date'] = pd.to_datetime(df_META_historical['Date'])
df_MSFT_historical['Date'] = pd.to_datetime(df_MSFT_historical['Date'])
df_NVDA_historical['Date'] = pd.to_datetime(df_NVDA_historical['Date'])
df_TSLA_historical['Date'] = pd.to_datetime(df_TSLA_historical['Date'])

# df_AAPL_historical['Date'] = pd.to_datetime(df_AAPL_historical['Date'], utc=True)

In [11]:
  # Set the dataframe index to the Date column instead of the default range
df_AAPL_historical.set_index('Date', inplace=True)

In [12]:
# Create a DataFrame with dates from the AAPL data
dates = df_AAPL_historical.index
text_data = pd.DataFrame(dates, columns=['Date'])

In [13]:
# Generate random headlines
headline_samples = [
    "AAPL hits record high",
    "Concerns over AAPL's future growth",
    "AAPL to unveil new product next month",
    "AAPL reports earnings that exceed forecasts",
    "Market downturn affects AAPL",
    "AAPL invests in renewable energy",
    "New AAPL CEO announced",
    "AAPL faces regulatory scrutiny",
    "AAPL rumored to acquire a tech startup",
    "AAPL's market share grows"
]

In [14]:
# Assign a random headline to each date
text_data['Headline'] = np.random.choice(headline_samples, size=len(text_data))

# Display the first few rows of the updated DataFrame
text_data.head()

,Date,Headline
0,1980-12-12,AAPL to unveil new product next month
1,1980-12-15,Market downturn affects AAPL
2,1980-12-16,Concerns over AAPL's future growth
3,1980-12-17,AAPL rumored to acquire a tech startup
4,1980-12-18,AAPL faces regulatory scrutiny


In [15]:
# Applying the sentiment analysis function to your headlines
text_data['Sentiment'] = text_data['Headline'].apply(SentimentAnalysis.calculate_sentiment)

text_data.head(30)

,Date,Headline,Sentiment
0,1980-12-12,AAPL to unveil new product next month,0.068182
1,1980-12-15,Market downturn affects AAPL,0.000000
2,1980-12-16,Concerns over AAPL's future growth,0.000000
3,1980-12-17,AAPL rumored to acquire a tech startup,0.000000
4,1980-12-18,AAPL faces regulatory scrutiny,0.000000
5,1980-12-19,AAPL hits record high,0.160000
6,1980-12-22,AAPL invests in renewable energy,0.000000
7,1980-12-23,AAPL rumored to acquire a tech startup,0.000000
8,1980-12-24,AAPL invests in renewable energy,0.000000
9,1980-12-26,AAPL invests in renewable energy,0.000000


In [16]:
# Aligning sentiment scores with the stock closing prices on the same date
combined_data = text_data.set_index('Date').join(df_AAPL_historical['Close'])
combined_data.head(30)

,Headline,Sentiment,Close
Date,,,
1980-12-12,AAPL to unveil new product next month,0.068182,0.128348
1980-12-15,Market downturn affects AAPL,0.000000,0.121652
1980-12-16,Concerns over AAPL's future growth,0.000000,0.112723
1980-12-17,AAPL rumored to acquire a tech startup,0.000000,0.115513
1980-12-18,AAPL faces regulatory scrutiny,0.000000,0.118862
1980-12-19,AAPL hits record high,0.160000,0.126116
1980-12-22,AAPL invests in renewable energy,0.000000,0.132254
1980-12-23,AAPL rumored to acquire a tech startup,0.000000,0.137835
1980-12-24,AAPL invests in renewable energy,0.000000,0.145089
